## Median Spread against Time to Expiry

A number of quantities become interesting as time to expiration nears, including:

1. number of trades

2. spread

  a. spread
  
  b. vwap
  

In [23]:
import pandas as pd
import numpy as np
import calendar
from datetime import datetime

import matplotlib.pyplot as plt
import mpld3

In [24]:
def btc_cme_expiration_time(tkr, timezone='UTC'):    
    assert tkr[:3] == 'BTC', f'{tkr} is not a CME bitcoin future'    
    d_codes = {'F':1,'G':2,'H':3,'J':4,'K':5,'M':6,'N':7,'Q':8,
               'U':9,'V':10,'X':11,'Z':12}
    d_year = {'9':2019,'0':2020}
    assert tkr[-2] in d_codes, f'Wrong ticker name format {tkr} for month code'
    assert tkr[-1] in d_year, f'Wrong ticker name format {tkr} for year code'    
    year = int(d_year[tkr[-1]])
    month = int(d_codes[tkr[-2]])
    # Last friday of the month
    day = max(week[calendar.FRIDAY] for week in calendar.monthcalendar(year, month))    
    t = pd.to_datetime(f'{year}-{month}-{day} 16:00:00')    
    return t.tz_localize('Europe/London').tz_convert(timezone)

# futures top of book

def fut_day_level(tkr):
    data_path = "../data_dumps"
    curncy_data = f"{data_path}/{tkr}_Curncy_top.csv"
    expiry = btc_cme_expiration_time(tkr)

    df = pd.read_csv(curncy_data, parse_dates=["time"])
    df = df.set_index('time')
    df.index = df.index.tz_localize('UTC')
    df["spread"] = np.abs(df["ask_price"] - df["bid_price"])
    df["days_to_expiry"] = (df.index - expiry).days
    
    return df

In [25]:
# proper time to expiry plot over all *0 futures contracts, pt.1

futures = '''BTCH0,BTCJ0,BTCK0,BTCM0,BTCN0,BTCQ0,BTCU0,BTCV0,BTCX0,BTCZ0'''.split(',')
agg_columns = "time,idx,bid_prc,bid_qty,ask_prc,ask_qty,trd_prc,trd_qty,spread_5,spread_25,spread_50,spread_75,spread_95,counts,days_to_expiry,fut_prc,expiry".split(',')
fut = []
for i in futures:
    df = fut_day_level(i)
    fd = df.groupby('days_to_expiry').median()
    fut.append(fd)

In [26]:
# proper time to expiry plot over all *0 futures contracts, pt.2
fig, ax = plt.subplots(figsize=(12,8))

for idx,frame in enumerate(fut):    
    ax.plot(frame.index, frame['spread'], label=futures[idx])
    
ax.set_title("2020 futures contracts against time to expiry")
ax.set_xlabel("time to expiry")
ax.set_ylabel("median spread")
ax.legend(loc="upper right")
plt.xlim(-195,-10)
plt.ylim(0,100)

ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%D"))    

mpld3.display(fig)